In [16]:
# Welcome to your new notebook
# Installing libraries
!pip install requests
!pip install beautifulsoup4
!pip install boto3 pandas




StatementMeta(, fe710341-eeac-4f00-8d47-816c64a8dc86, 18, Finished, Available)

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

StatementMeta(, fe710341-eeac-4f00-8d47-816c64a8dc86, 19, Finished, Available)

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of URLs to scrape
urls = [
    {"url": "https://www.skysports.com/africa-cup-of-nations-table", "source": "Sky Sports"}
    # Add more URLs as needed
]

# Create an empty list to store DataFrames
dfs = []

for url_info in urls:
    url = url_info["url"]
    source = url_info["source"]

    # Send HTTP Request and Parse HTML
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")

    # Find and Extract all tables with the specified class name
    tables = soup.find_all("table", class_="standing-table__table")

    for table in tables:
        # Find and Extract Table Headers
        headers = table.find_all("th")
        titles = [i.text for i in headers]

        # Create an Empty DataFrame
        df = pd.DataFrame(columns=titles)

        # Iterate Through Table Rows and Extract Data
        rows = table.find_all("tr")
        for i in rows[1:]:
            data = i.find_all("td")
            row = [tr.text.strip() for tr in data]  # Apply .strip() to remove \n
            l = len(df)
            df.loc[l] = row

        # Add a column for source URL
        df["Source"] = source

        #Add Timestamp
        df["Refresh_Timestamp"] = pd.to_datetime(datetime.now())

        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
football_standing = pd.concat(dfs, ignore_index=True)

# Remove first two columns
football_standing = football_standing.iloc[:, 1:]

# Print the merged DataFrame
print(football_standing)


StatementMeta(, fe710341-eeac-4f00-8d47-816c64a8dc86, 20, Finished, Available)

                 Team Pl  W  D  L  F  A  GD Pts Last 6      Source  \
0         Ivory Coast  1  1  0  0  2  0   2   3         Sky Sports   
1   Equatorial Guinea  1  0  1  0  1  1   0   1         Sky Sports   
2             Nigeria  1  0  1  0  1  1   0   1         Sky Sports   
3       Guinea-Bissau  1  0  0  1  0  2  -2   0         Sky Sports   
4          Cape Verde  1  1  0  0  2  1   1   3         Sky Sports   
5               Egypt  1  0  1  0  2  2   0   1         Sky Sports   
6          Mozambique  1  0  1  0  2  2   0   1         Sky Sports   
7               Ghana  1  0  0  1  1  2  -1   0         Sky Sports   
8             Senegal  1  1  0  0  3  0   3   3         Sky Sports   
9            Cameroon  1  0  1  0  1  1   0   1         Sky Sports   
10             Guinea  1  0  1  0  1  1   0   1         Sky Sports   
11             Gambia  1  0  0  1  0  3  -3   0         Sky Sports   
12            Burkina  1  1  0  0  1  0   1   3         Sky Sports   
13            Algeri

In [19]:
# Declare file name    
file_name = "Afcon_raw"

# Construct destination path
output_parquet_path = f"abfss://AFCON@onelake.dfs.fabric.microsoft.com/AFCON_Lake.Lakehouse/Files/Raw_Data/{file_name}.parquet"

# Save DataFrame to Parquet file
football_standing.to_parquet(output_parquet_path, index=False)

print(f"DataFrame saved to Parquet file: {output_parquet_path}")


StatementMeta(, fe710341-eeac-4f00-8d47-816c64a8dc86, 21, Finished, Available)

DataFrame saved to Parquet file: abfss://AFCON@onelake.dfs.fabric.microsoft.com/AFCON_Lake.Lakehouse/Files/Raw_Data/Afcon_raw.parquet


In [ ]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'Football'

df = spark.read.format("parquet").load('Files/wwi-raw-data/full/fact_sale_1y_full')
df = df.withColumn('Quarter', quarter(col("InvoiceDateKey")))
df = df.withColumn('Month', month(col("InvoiceDateKey")))

df.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)